# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [3]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [6]:
colunas_necessarias = {'munResNome', 'IDADEMAE', 'IDADEPAI'}
colunas_necessarias.issubset(sinasc_raw.columns)

df_filtrado = sinasc_raw[(sinasc_raw['IDADEMAE'].notna()) & (sinasc_raw['IDADEPAI'].notna())]

idades_medias = df_filtrado.groupby('munResNome')[['IDADEMAE', 'IDADEPAI']].mean().reset_index()

idades_medias.columns = ['Município de Residência', 'Idade Média da Mãe', 'Idade Média do Pai']

print(idades_medias.head(10))

   Município de Residência  Idade Média da Mãe  Idade Média do Pai
0    Alta Floresta D'Oeste           25.892857           29.548701
1  Alto Alegre dos Parecis           24.679104           29.179104
2             Alto Paraíso           22.833333           28.833333
3         Alvorada D'Oeste           26.825243           30.757282
4                Ariquemes           28.133333           32.466667
5                  Buritis           24.750000           30.812500
6                   Cabixi           32.166667           34.333333
7              Cacaulândia           31.000000           36.333333
8                   Cacoal           26.978605           30.931854
9   Campo Novo de Rondônia           24.900000           30.500000


### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [7]:
sinasc_raw['DTNASC'] = pd.to_datetime(sinasc_raw['DTNASC'], errors='coerce')

nascidos_2903 = sinasc_raw[
    (sinasc_raw['DTNASC'].dt.month == 3) &
    (sinasc_raw['DTNASC'].dt.day == 29)
]

nascidos_2903 = nascidos_2903[['SEXO', 'ESCMAE', 'PESO']].dropna()

peso_medio = nascidos_2903.groupby(['SEXO', 'ESCMAE'])['PESO'].mean().reset_index()

print(peso_medio)

        SEXO           ESCMAE         PESO
0   Feminino  12 anos ou mais  2982.500000
1   Feminino       4 a 7 anos  3297.875000
2   Feminino      8 a 11 anos  3206.600000
3  Masculino  12 anos ou mais  3311.250000
4  Masculino       4 a 7 anos  2896.875000
5  Masculino      8 a 11 anos  3514.678571


### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [21]:
df_filtrado = sinasc_raw[sinasc_raw['munResNome'] != 'Município ignorado - RO']

nascimentos_por_municipio = df_filtrado['munResNome'].value_counts()

municipio_menos_nascimentos = nascimentos_por_municipio.idxmin()
print(f"Município com menos nascimentos: {municipio_menos_nascimentos}")

dados_municipio = df_filtrado[df_filtrado['munResNome'] == municipio_menos_nascimentos]

idade_maes = dados_municipio['IDADEMAE']
media_maes = idade_maes.mean()
max_mae = idade_maes.max()
min_mae = idade_maes.min()

print(f"\nIdade das mães em {municipio_menos_nascimentos}:")
print(f"Média: {media_maes:.2f}")
print(f"Máxima: {max_mae}")
print(f"Mínima: {min_mae}")

idade_pais = dados_municipio['IDADEPAI'].dropna()
media_pais = idade_pais.mean()
max_pai = idade_pais.max()
min_pai = idade_pais.min()

print(f"\nIdade dos pais em {municipio_menos_nascimentos}:")
print(f"Média: {media_pais:.2f}")
print(f"Máxima: {max_pai}")
print(f"Mínima: {min_pai}")


Município com menos nascimentos: Castanheiras

Idade das mães em Castanheiras:
Média: 27.28
Máxima: 39
Mínima: 17

Idade dos pais em Castanheiras:
Média: 30.39
Máxima: 43.0
Mínima: 17.0


### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



In [8]:
nasc_marco = sinasc_raw[sinasc_raw['DTNASC'].dt.month == 3]

nascimentos_marco = nasc_marco['munResNome'].value_counts()

municipio_top = nascimentos_marco.idxmax()
quantidade = nascimentos_marco.max()

print(f"Município com mais nascimentos em março: {municipio_top}")
print(f"Quantidade de nascimentos: {quantidade}")

Município com mais nascimentos em março: Porto Velho
Quantidade de nascimentos: 744


### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

Exemplo:
- Ah, descobri que a idade mediana das mulheres que deram a luz no ano de 2019 dos municipios x é maior que y.

In [ ]:
#Notei e achei interessante que a idade miníma dos pais e mães no município que menos nasceu bebês é a mesma, ou seja 17 anos.

In [6]:
sinasc_raw.dtypes

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResUf       object
munResLat     float64
munResLon     float64
munResAlt     float64
munResArea    float64
Length: 69, dtype: object

In [7]:
sinasc_raw.select_dtypes(exclude=['int64','float64'])

,ESCMAE,GESTACAO,GRAVIDEZ,PARTO,DTNASC,SEXO,RACACOR,DTCADASTRO,CODANOMAL,VERSAOSIST,DTRECEBIM,munResStatus,munResTipo,munResNome,munResUf
0,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-19,Masculino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
1,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-21,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
2,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-25,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
3,12 anos ou mais,37 a 41 semanas,Única,Cesáreo,2019-03-20,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia
4,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-03-23,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,12 anos ou mais,32 a 36 semanas,Única,Cesáreo,2019-12-13,Masculino,Branca,2020-09-28,NaN,3.2.01,2020-10-13,ATIVO,MUNIC,Vilhena,Rondônia
27024,8 a 11 anos,37 a 41 semanas,Única,Vaginal,2019-10-04,Masculino,Branca,2019-10-16,NaN,3.2.01,2019-10-21,ATIVO,MUNIC,Chupinguaia,Rondônia
27025,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-08-02,Masculino,Parda,2019-08-07,NaN,3.2.00,2019-08-07,ATIVO,MUNIC,Vilhena,Rondônia
27026,8 a 11 anos,32 a 36 semanas,Única,Vaginal,2019-12-23,Masculino,Parda,2020-01-07,NaN,3.2.00,2020-01-07,ATIVO,MUNIC,Vilhena,Rondônia


In [4]:
sinasc_raw.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')